In [23]:
# 패키지 설치 (필요한 경우)
%pip install langchain-upstage python-dotenv


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: /opt/homebrew/opt/python@3.11/bin/python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [24]:
# 필요한 라이브러리 import
import os
from dotenv import load_dotenv
from langchain_upstage import ChatUpstage

## 환경 변수 로드

In [25]:
# .env 파일에서 환경 변수 불러오기
load_dotenv()

# Upstage API 키 확인
api_key = os.getenv("UPSTAGE_API_KEY")
if not api_key:
    print("⚠️ UPSTAGE_API_KEY가 설정되지 않았습니다. .env 파일을 확인하세요.")
else:
    print("✅ API 키가 로드되었습니다.")

✅ API 키가 로드되었습니다.


## 솔라프로2 호출 테스트

In [26]:
# 솔라프로2 모델 초기화
chat = ChatUpstage(
    model="solar-pro2-251215",
    upstage_api_key=api_key
)

# "헬로우" 메시지 전송
response = chat.invoke("헬로우")

print("응답:", response.content)

응답: 안녕하세요! 😊  

"헬로우"라고 하셨네요! 영어로는 "Hello"인 것 같아요. 한국어로는 "안녕하세요"라고 합니다.  

무엇을 도와드릴까요? 😊  
(예: 질문, 대화, 정보 요청 등 자유롭게 말씀해 주세요!)  

* 오타가 있으셨다면 수정도 가능합니다! 😉*


## LangGraph

In [ ]:
### State 정의
from typing import Literal, TypedDict
from langgraph.graph import StateGraph, END
# State 타입 정의
class WorkflowState(TypedDict):
    messages: list
    # 필요한 다른 상태 변수들을 여기에 추가
    pass

In [28]:
### 노드 함수 정의

def ingest(state: WorkflowState) -> WorkflowState:
    print("ingest")
    return state

def extract_emotion(state: WorkflowState) -> WorkflowState:
    print("extract_emotion")
    return state

def decide(state: WorkflowState) -> WorkflowState:
    print("decide")
    return state

def plan_noti(state: WorkflowState) -> WorkflowState:
    print("plan_noti")
    return state

def send_noti(state: WorkflowState) -> WorkflowState:
    print("send_noti")
    return state

def eval_noti(state: WorkflowState) -> WorkflowState:
    print("eval_noti")
    return state

def write_report(state: WorkflowState) -> WorkflowState:
    print("write_report")
    return state

def eval_report(state: WorkflowState) -> WorkflowState:
    print("eval_report")
    return state

In [29]:
### 라우팅 함수 정의

from typing import Literal

def route_after_decide(state: WorkflowState) -> Literal["write_report", "plan_noti"]:
    return "plan_noti"

def route_after_plan_noti(state: WorkflowState) -> Literal["send_noti", "__end__"]:
    return "send_noti"

def route_after_eval_noti(state: WorkflowState) -> Literal["send_noti", "__end__"]:
    return "__end__"   # retry 없이 종료

def route_after_eval_report(state: WorkflowState) -> Literal["write_report", "__end__"]:
    return "__end__"

In [30]:
### 그래프 구성

# StateGraph 생성
workflow = StateGraph(WorkflowState)

# 노드 추가
workflow.add_node("ingest", ingest)
workflow.add_node("extract_emotion", extract_emotion)
workflow.add_node("decide", decide)
workflow.add_node("plan_noti", plan_noti)
workflow.add_node("send_noti", send_noti)
workflow.add_node("eval_noti", eval_noti)
workflow.add_node("write_report", write_report)
workflow.add_node("eval_report", eval_report)

# 엣지 추가 (순차 흐름)
workflow.set_entry_point("ingest")
workflow.add_edge("ingest", "extract_emotion")
workflow.add_edge("extract_emotion", "decide")

# 조건부 라우팅
workflow.add_conditional_edges(
    "decide",
    route_after_decide,
    {
        "write_report": "write_report",
        "plan_noti": "plan_noti"
    }
)

workflow.add_conditional_edges(
    "plan_noti",
    route_after_plan_noti,
    {
        "send_noti": "send_noti",
        "__end__": END
    }
)

workflow.add_edge("send_noti", "eval_noti")

workflow.add_conditional_edges(
    "eval_noti",
    route_after_eval_noti,
    {
        "send_noti": "send_noti",
        "__end__": END
    }
)

workflow.add_edge("write_report", "eval_report")

workflow.add_conditional_edges(
    "eval_report",
    route_after_eval_report,
    {
        "write_report": "write_report",
        "__end__": END
    }
)

# 그래프 컴파일
app = workflow.compile()
# from IPython.display import Image, display

# g = app.get_graph()
# png_bytes = g.draw_mermaid_png()
# display(Image(png_bytes))

print("✅ LangGraph 워크플로우가 생성되었습니다!")

✅ LangGraph 워크플로우가 생성되었습니다!


In [31]:
### 워크플로우 테스트

# 초기 state 생성
initial_state = WorkflowState(messages=[])

# 워크플로우 실행
result = app.invoke(initial_state)

print("\n✅ 워크플로우 실행 완료!")
print(f"최종 state: {result}")

ingest
extract_emotion
decide
plan_noti
send_noti
eval_noti

✅ 워크플로우 실행 완료!
최종 state: {'messages': []}


In [35]:
messages = [
    "아 오늘 회의 사장 개빡침",
    "야식 먹어서 살찌겟네 ㅠ",
    "내일 친구 5시 고깃집",
    "내일 회의 있음",
]

# 초기 state에 messages 포함
initial_state = WorkflowState(messages=messages)

# 워크플로우 실행
result = app.invoke(initial_state)

print("\n✅ 워크플로우 실행 완료!")
print(f"최종 state: {result}")

ingest
extract_emotion
decide
plan_noti
send_noti
eval_noti

✅ 워크플로우 실행 완료!
최종 state: {'messages': ['아 오늘 회의 사장 개빡침', '야식 먹어서 살찌겟네 ㅠ', '내일 친구 5시 고깃집', '내일 회의 있음']}
